In [1]:
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Lambda, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Model, Sequential
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import LSTM



Using TensorFlow backend.


In [2]:
#Loading the file
df=pd.read_csv('C:/My Backup/Practice Folder/Insofe Course/Material/Cute Test Preparation/Test 4/cse7321c-cute04-team-8_john_sai-_narayan-master/data.csv')


In [3]:
#Dropping all rows with blanks in column converse
df=df.dropna(axis=0, how='any')


In [4]:
#Dropping all rows with JUNK in column categories
df = df[df["categories"] != 'JUNK']

In [5]:
#Dropping all rows with JUNK in column sub_categories
df = df[df["sub_categories"] != 'JUNK']

In [6]:
docs2=df["converse"].tolist()

In [7]:
print(df.shape)

(57234, 4)


In [8]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs2)
vocab_size = len(t.word_index) + 1

In [9]:
# integer encode the documents
encoded_docs2 = t.texts_to_sequences(docs2)



In [12]:
# pad documents to a max length of 1000 words
max_length = 1000
padded_docs = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
print(padded_docs)

#labels=to_categorical(np.asarray(df.categories.tolist()))
labels = pd.get_dummies(df['sub_categories'])
print('Shape of data tensor:', padded_docs.shape)
print('Shape of label tensor:', labels.shape)


[[   2  410   18 ...,    0    0    0]
 [  31  118    1 ...,    0    0    0]
 [   2    1  120 ...,    0    0    0]
 ..., 
 [1985 1165   22 ...,    0    0    0]
 [4319 1549   39 ...,    0    0    0]
 [ 240    7 1115 ...,    0    0    0]]
Shape of data tensor: (57234, 1000)
Shape of label tensor: (57234, 20)


In [13]:
df_combine=np.concatenate((padded_docs, labels), axis=1)

In [14]:
num_validation_samples = int(0.2 * padded_docs.shape[0])
num_validation_samples

11446

In [68]:
#filecombine = pd.DataFrame(df_combine).to_csv('C:\\My Backup\\Practice Folder\\Insofe Course\\Material\\Cute Test Preparation\\Test 4\\cse7321c-cute04-team-8_john_sai-_narayan-master\\df_combine.csv')

In [15]:
#Splitting into train & validation
num_validation_samples = int(0.2 * padded_docs.shape[0])

x_train = padded_docs[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = padded_docs[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

Preparing embedding matrix.


In [44]:
#filextrain = pd.DataFrame(x_train).to_csv('C:\\My Backup\\Practice Folder\\Insofe Course\\Material\\Cute Test Preparation\\Test 4\\cse7321c-cute04-team-8_john_sai-_narayan-master\\xtraintemp.csv')
#filexval = pd.DataFrame(x_val).to_csv('C:\\My Backup\\Practice Folder\\Insofe Course\\Material\\Cute Test Preparation\\Test 4\\cse7321c-cute04-team-8_john_sai-_narayan-master\\xval.csv')
#fileytrain = pd.DataFrame(y_train).to_csv('C:\\My Backup\\Practice Folder\\Insofe Course\\Material\\Cute Test Preparation\\Test 4\\cse7321c-cute04-team-8_john_sai-_narayan-master\\ytraintemp.csv')
#fileyval = pd.DataFrame(y_val).to_csv('C:\\My Backup\\Practice Folder\\Insofe Course\\Material\\Cute Test Preparation\\Test 4\\cse7321c-cute04-team-8_john_sai-_narayan-master\\yval.csv')

In [16]:
embeddings_index = dict()
f = open('C:/My Backup/Practice Folder/Insofe Course/Material/Day 29/glove.6B/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [17]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [22]:
#Building a RNN model
print('Build model...')
model = Sequential()

model.add(Embedding(vocab_size,
                    100,
                    input_length=1000,
                    weights=[embedding_matrix],
                    trainable=False
                    ))
#model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='softmax'))


Build model...


In [23]:
from keras import callbacks
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=5, min_lr=0.00001, verbose=1, epsilon=0.001)
early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=1, mode='auto')
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [24]:
model.fit(np.array(x_train), np.array(y_train), batch_size=64, epochs=1, validation_data=(np.array(x_val), np.array(y_val)), callbacks=[reduce_lr, early_stop])

Train on 45788 samples, validate on 11446 samples
Epoch 1/1
  448/45788 [..............................] - ETA: 1410s - loss: 2.9604 - acc: 0.1674

KeyboardInterrupt: 

In [ ]:
# summarize the model
print(model.summary())